<img src='img/LogoWekeo_Copernicus_RGB_0.png' alt='' align='centre' width='10%'></img>

### <a id='wekeo_hda_install'></a>Introduction


<h4>This Jupyter notebook includes examples on:</h4>
<li>How to access GFSC products through HDA API</li>
<li>Read and visualize GFSC Products</li>
<li>Manipulate QCFLAGS layer to filter data according to sensor type</li>
<li>Manipulate AT layer to filter data according to data age</li>

<h4>CLMS HRSI GFSC Product:</h4>
The daily cumulative Gap-filled Fractional Snow Cover (GFSC) product is generated in NRT for
the entire EEA38+UK domain based on SAR data from the S1 constellation and optical data
from the S2 constellation. The product merges the latest observations available to form a
spatially complete overview of snow conditions. The product provides the extent of the snow
cover per pixel as a percentage (0% – 100%) with a spatial resolution of 60 m x 60 m. The
product uses Fractional Snow Cover (FSC), Wet/Dry Snow (WDS) and SAR Wet Snow (SWS) products as input to form a spatially complete composite of
snow conditions, to reduce observational gaps due to clouds and lack of sensor coverage on a
daily basis. The product applies the on-ground FSC, and SWS and
presents the combined information as FSC.
<br><br>
Additional information about GFSC product can be found in the <a href="https://land.copernicus.eu/user-corner/technical-library/hrsi-snow-pum">Product User Manual (PUM).</a>

<h4>Importing  the required libraries</h4>

In [ ]:
# Libraries and definitions
import os, sys, json, datetime, dateutil, shutil, tempfile, zipfile
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from copy import deepcopy
prod_dir = os.path.realpath("products")

HDA API Python library is available by default with "miniwekeolab" kernel, but not with other kernels. We can make an exception to install it, if it does not exist.

In [ ]:
try:
    # hda library is not in Python 3 kernel (if selected)
    import hda
except:
    print("Cannot import hda, installing.")
    os.system("pip install hda")
    try:
        import hda
        print("Successful")
    except:
        print("Cannot import")


"xmltodict" Python library is not available by default with "miniwekeolab" kernel. We can make an exception to install it, if it does not exist.

In [ ]:
try:
    # xmltodict library is not in miniwekeolab kernel (if selected)
    import xmltodict
except:
    print("Cannot import xmltodict, installing.")
    os.system("pip install xmltodict")
    try:
        import xmltodict
        print("Successful")
    except:
        print("Cannot import")

<h4>Accessing GFSC Products using HDA Library</h4>
Harmonized data access (HDA) client can be used to access and download CLMS HRSI products. How to use the client is described in <a href="https://www.wekeo.eu/docs/how-to-use-hda">Wekeo documentation.</a> How to use HDA Python library is also documented </a href="https://hda.readthedocs.io/en/latest/">in a separate documentation</a> and a smaller tutorial can be found <a href="https://www.wekeo.eu/docs/hda-python-lib">here.</a> Wekeo also maintains a larger tutorial using Jupyter notebook <a href="https://github.com/wekeo/wekeo4data">in Github</a>. This notebook is also provided in the public folder of Wekeo Github at "public/wekeo4data/wekeo-hda/wekeo_harmonised_data_access_api.ipynb".
<br><br>
In this notebook, we will use HDA Python library with the default credentials file (~/.hdarc).

Create ~/.hdarc file and enter credentials if it doesnt exist: (replace "username" and "password" with correct ones)

In [ ]:
if not os.path.exists(os.path.join(os.path.expanduser('~'),'.hdarc')):
    with open(os.path.join(os.path.expanduser('~'),'.hdarc'),"w") as f:
        f.write("url: https://wekeo-broker.prod.wekeo2.eu/databroker\n")
        f.write("user: username\n")
        f.write("password: password\n")
    print("Credentials file created")
else:
    print("Credentials file already exists")

Initialize an HDA Client. Here, the user credentials are read from "~/.hdarc" file and default configuration is used.

In [ ]:
hda_client = hda.Client()

An example query JSON (will not be used in the examples) to search products:

In [ ]:
query = {
  "datasetId": "EO:CRYO:DAT:HRSI:GFSC",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        4.0488399662761605,
        60.64709920038568,
        9.213189474039462,
        61.81526952644772
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "Observed",
      "start": "2022-03-13T00:00:00.000Z",
      "end": "2022-03-13T00:00:00.000Z"
    }
  ]
}

Query JSON can be copy pasted from a manual search in Wekeo Data Explorer:
<div style='text-align:center;'>
<figure><img src='./img/apicall.png' width='50%' />
    <figcaption><i>S2 Tile T32VMN</i></figcaption>
</figure>
</div>

In [ ]:
query = {
  "datasetId": "EO:CRYO:DAT:HRSI:GFSC",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        34.871454195608834,
        38.98657296199103,
        36.45614160450388,
        39.91031559266836
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "Observed",
      "start": "2022-02-11T00:00:00.000Z",
      "end": "2022-02-15T00:00:00.000Z"
    }
  ],
  "stringInputValues": [
    {
      "name": "cloudCover",
      "value": "30"
    }
  ]
}

<table>
    <tr><td>
    We can create a query for a single day and tile, using "dateRangeSelectValues" and "stringInputValues" options.
        <br><br>
    For example, T32TVM in Scandinavian mountains, on 13.03.2022:
    </td><td>
<div style='text-align:center;'>
<figure><img src='./img/T32VMN.png' width='50%' />
    <figcaption><i>S2 Tile T32VMN</i></figcaption>
</figure>
</div>
        </td></tr>
    </table>

In [ ]:
# query by tile and date
tile = 'T32VMN'
day = '14.03.2022'

query = {
  "datasetId": "EO:CRYO:DAT:HRSI:GFSC",
  "dateRangeSelectValues": [
    {
      "name": "Observed",
      "start": datetime.datetime.strptime(day,"%d.%m.%Y").strftime("%Y-%m-%dT00:00:00.000Z"),
      "end": datetime.datetime.strptime(day,"%d.%m.%Y").strftime("%Y-%m-%dT00:00:00.000Z"),
    }
  ],
  "stringInputValues": [
    {
      "name": "productIdentifier",
      "value": tile
    }
  ]
}

Search the database with HDA Client and the query. Use a shorter timeout configuration, in case there is a problem in the connection or account.

In [ ]:
hda_client.time_sleep = 10
hda_client.timeout = 10

try:
    matches = hda_client.search(query)
    hda_success = True
except Exception as e:
    hda_success = False
    print(e)

If a problem occured, e.g. the user is not registered, create an object to use if there is any products downloaded before.

In [ ]:
if not hda_success:
    matches = []
    for product in os.listdir(prod_dir):
        try:
            # test if folder name fits the product filename format
            day = datetime.datetime.strptime(product.split('_')[1].split('-')[0],"%Y%m%d")
            curation_time = datetime.datetime.utcfromtimestamp(int(product.split('_')[5]))
            # add product name to the object in the form that it can be used in the line later
            # matches[0].results[0]['productInfo']['product']
            match = lambda: None
            match.results = [{'productInfo':{'product':product}}]
            matches.append(match)
        except:
            print("A folder/file in products folder is not a valid product. Skipped.")
    print(len(matches), "products found in products folder.")
else:
    print("HDA Client response successful. HDA Client will be used.")

Parse the response from HDA client and create a list of dictionaries with additional metadata to handle the data more compactly.
<br>
List structure:

In [ ]:
[
    {
        'title': str, 
        'tile': str, 
        'day': datetime.datetime, 
        'curation_time': datetime.datetime, 
        'dir': str
    },
    {
        'title': str, 
        'tile': str, 
        'day': datetime.datetime, 
        'curation_time': datetime.datetime, 
        'dir': str
    }
];

# only for descriptive usage. this cell does not do anything.

In [ ]:
print(len(matches), "products found")
# reparse response for a custom list
products = []
for match in matches:        
    product = match.results[0]['productInfo']['product']
    tile = product.split('_')[3]
    day = datetime.datetime.strptime(product.split('_')[1].split('-')[0],"%Y%m%d")
    curation_time = datetime.datetime.utcfromtimestamp(int(product.split('_')[5]))
    products.append({
        'title':product,
        'tile':tile,
        'day':day,
        'curation_time':curation_time
    })
    
# summarize products 
print("Day\t\tTile\tCuration time")
for product in products:
    print(
        '\t'.join([
            str(product['day']).split()[0],product['tile'],str(product['curation_time'])
        ])
    )
    
if products == []:
    print("No products are available in the disk. Rest of the notebook will not work.")

Download the products to temporary folders and move them to the "products directory" which was defined in the beginning.
<li>Products are downloaded as zip files, but without file extensions. Extract the files before moving them to the products directory.</li>
<li>Products may be downloaded before, if that is the case (folder exists), skip the download.</li>

In [ ]:
os.makedirs(prod_dir,exist_ok=True)
for product in products:
    if product['title'] not in os.listdir(prod_dir):
        # use a temporary directory
        with tempfile.TemporaryDirectory() as tmp_dir:
            # find the products in the HDA client response and call download method
            [match.download(download_dir = tmp_dir) for match in matches if match.results[0]['productInfo']['product'] == product['title'] ]
            # rename file to zip to use same dir for product dir, to be safe in real product folder
            os.rename(os.path.join(tmp_dir,product['title']), os.path.join(tmp_dir,product['title']) + '.zip')
            # unzip
            with zipfile.ZipFile(os.path.join(tmp_dir,product['title']) + '.zip') as zf:
                zf.extractall(os.path.join(tmp_dir))
            # copy to the "real" product dir
            shutil.copytree(os.path.join(tmp_dir,product['title']),os.path.join(prod_dir,product['title']))
            print(product['title'],"was downloaded to",prod_dir)
    else:
        print(product['title'],"is already in product directory")
    # add directory information to be used later
    product['dir'] = os.path.join(prod_dir,product['title'])

# Note: HDA Client can download all results at once:
# matches.download(download_dir = path_to_directory)

<h4>Read and visualize products</h4>
Create functions to read products rasters and metadata and keep them compactly in dictionaries:
<br><br>
Main function will get the directory and the title of the product as arguments and return a single dictionary with all data.

In [ ]:
# function to read tif image
def readTif(file_tif):

    ds = gdal.Open(file_tif)
    # Projection
    proj = ds.GetProjection()
    # Raster data
    data = ds.GetRasterBand(1).ReadAsArray()
    # colormap
    color_table = ds.GetRasterBand(1).GetRasterColorTable()
    cmp = None
    if color_table is not None:
        cmp = []
        for i in range(color_table.GetCount()):
            color = color_table.GetColorEntry(i)
            cmp.append((color[0]/255, color[1]/255, color[2]/255))
    ds = None
    return data, proj, cmp

# function to read all rasters and xml metadata of gfsc
def readProductFiles(directory,title):   
    # read tif rasters
    gf, proj_gf, cmp_gf = readTif(os.path.join(directory,title) + '_GF.tif')
    qc, proj_qc, cmp_qc = readTif(os.path.join(directory,title) + '_QC.tif')
    qcflags = readTif(os.path.join(directory,title) + '_QCFLAGS.tif')[0]
    at = readTif(os.path.join(directory,title) + '_AT.tif')[0]
    
    # read xml metadata
    meta = xmltodict.parse(open(os.path.join(directory,title) + '_MTD.xml','r').read())
    
    return gf, cmp_gf, qc, cmp_qc, qcflags, at, meta, proj_gf

# function to organize data into single dictonary
def readProductData(directory,title):
    gf, cmp_gf, qc, cmp_qc, qcflags, at, meta, proj = readProductFiles(directory,title)
    
    # make a dict for convenient reading
    productData = {
        'GF':gf, 
        'GF_cmp':cmp_gf, 
        'QC':qc, 
        'QC_cmp':cmp_qc, 
        'QCFLAGS':qcflags, 
        'AT':at, 
        'meta':meta, 
        'proj':proj
    }
    
    return productData

Read data and metadata of the first product in the list using the function and directories defined previously

In [ ]:
product = products[0]
productData = readProductData(product['dir'],product['title'])

The color table in the product includes No data label (255) as transparent. Since the notebook color is white, no data pixels will be shown as white, which will not be distinguisable from snow pixels. Thus, we should use a different color for no data pixels:

In [ ]:
productData['GF_cmp'][-1] = (255/255.,192/255.,203/255.)

Default figure size of "matplotlib" is too small for the screen, modify it:

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

Draw GF layer using matplotlib:

In [ ]:
plt.imshow(productData['GF'], cmap = ListedColormap(productData['GF_cmp']), interpolation = 'nearest');
plt.clim(0, 255)
plt.colorbar();
plt.xticks([]);
plt.yticks([]);

GF layer shows gap-filled FSC. It contains data from up to 7 days before. XML metadata in the product includes the metadata information from all the input products. We can use that information to list the input products and their sensing times and calculate the age of data for each product, relatively to the GFSC product.

In [ ]:
print('TYPE\tSENSING TIME\t\tAGE')
for input_meta in productData['meta']['gmd:MD_Metadata']['gmd:series']['gmd:DS_OtherAggregate']['gmd:seriesMetadata']:
    input_title = input_meta['gmd:MD_Metadata']['gmd:fileIdentifier']['gco:CharacterString']
    input_type = input_title.split('_')[0]
    input_startDate = dateutil.parser.isoparse(input_meta['gmd:MD_Metadata']['gmd:identificationInfo']['gmd:MD_DataIdentification']['gmd:extent']['gmd:EX_Extent']['gmd:temporalElement']['gmd:EX_TemporalExtent']['gmd:extent']['gml:TimePeriod']['gml:beginPosition'])
    input_startDate = input_startDate.replace(microsecond=0)
    input_age = product['day'] + datetime.timedelta(days=1) - input_startDate
    print(input_type + '\t' + input_startDate.strftime('%Y-%m-%d %H:%M:%S') + '\t' + str(input_age))



XML metadata has the temporal information as a list, but AT layer has the temporal information for each pixel. Sensing time of each pixel is encoded as "Unix time" (seconds from 1.1.1970) in AT layer. We can create a raster of "age" which shows how old is the data in days (decimal), from the end of the product day (e.g. the age for a sensing at noon in the same day will be 12 hours).

In [ ]:
product_midnight_timestamp = datetime.datetime.timestamp(product['day'] + datetime.timedelta(days=1))
productData['age'] = (product_midnight_timestamp - productData['AT'])/(60*60*24.)

When no-data is present, AT layer has the value "0". To handle that, we can use "NaN" in the new age raster instead.

In [ ]:
np.place(productData['age'],productData['AT'] == 0,np.nan)

Draw the new "age" raster using matplotlib:

In [ ]:
plt.imshow(productData['age'],  cmap = 'tab20b', interpolation = 'nearest');
plt.colorbar();
plt.xticks([]);
plt.yticks([]);

<h4>Manipulating AT layer to filter out data according to age of the pixels</h4>
Using the age raster, we can filter out pixels which are "too old" for us. Create a new GF raster with only the pixels with "age" not more than 4 days (Max 3 days old data). 
<li>"invalid value encountered" warning will appear due to np.nan in age, but result is false for those pixels. Thus, they won't disturb the result.

In [ ]:
gf_4days = deepcopy(productData['GF'])
np.place(gf_4days, productData['age'] > 4, 255)

plt.subplot(1, 3, 1)
plt.title("GF")
plt.imshow(
    productData['GF'], cmap = ListedColormap(productData['GF_cmp']), interpolation = 'nearest'
);
plt.clim(0, 255)
plt.xticks([]);
plt.yticks([]);
# change the color to a different one
plt.subplot(1, 3, 2)
plt.title("4 Days Mask")
plt.imshow(
    productData['age'] <= 4, cmap = 'PiYG', interpolation = 'nearest'
);
plt.xticks([]);
plt.yticks([]);

plt.subplot(1, 3, 3)
plt.title("GF 4 Days")
plt.imshow(
    gf_4days, cmap = ListedColormap(productData['GF_cmp']), interpolation = 'nearest'
);
plt.clim(0, 255)
plt.xticks([]);
plt.yticks([]);


<h4>Manipulating QCFLAGS layer to filter out data according to sensor type of the pixels</h4>
QCFLAGS layer is encoded bitwise. It has information for 7 different cases as a combination. Thus, only reading integer values directly from the raster will not make sense.
Draw QCFLAGS raster using matplotlib:

In [ ]:
plt.title("QCFLAGS")
plt.imshow(
    productData['QCFLAGS'], cmap = 'Greys', interpolation = 'nearest'
);
plt.xticks([]);
plt.yticks([]);
plt.colorbar();

We should read the raster bit by bit, and use the information according to <a href="https://land.copernicus.eu/user-corner/technical-library/hrsi-snow-pum">the product user manual:</a>

➔ bit 0: sun elevation angle too
low for an accurate
topographic correction (from
MAJA) (active=1) (only valid if
bit 6 is 0)

➔ bit 1: solar elevation angle
tangent to slope (from MAJA)
(active=1) (only valid if bit 6
is 0)

➔ bit 2: water (from EU-Hydro)
(active=1)

➔ bit 3: TCD too high for
accurate forest correction
(TCD>90%) (active=1) (only
valid if bit 6 is 0)

➔ bit 4: snow detected under
thin clouds (active=1) (only
valid if bit 6 is 0)

➔ bit 5: TCD not defined or not
available (active=1) (only
valid if bit 6 is 0)

➔ bit 6: sensor type of the
satellite data (optical=0,
radar=1)

Create a function to use bit operations to read a single bit as a boolean raster:

In [ ]:
def getBit(data,bit):
    # shift the bits to the right until the bit in question is in first digit
    mask = np.right_shift(data,bit)
    # apply "AND 00000001" operation to get the value as 0 or 1.
    mask = np.bitwise_and(mask,1)
    return mask.astype(np.bool)

# Google "nth bit of a number" for the theory

Read all the bits and visualize them using matplotlib:

In [ ]:
for bit in range(1,7):
    plt.subplot(2, 3, bit)
    plt.title("Bit " + str(bit))
    plt.imshow(
        getBit(productData['QCFLAGS'],bit), cmap = 'binary', interpolation = 'nearest'
    );
    plt.xticks([]);
    plt.yticks([]);

Another example to read QCFLAGS raster can be found in "Webinar #2 (13 Oct. 2022): High Resolution Snow and Ice Monitoring" at <a href="https://land.copernicus.eu/pan-european/biophysical-parameters/high-resolution-snow-and-ice-monitoring/user-section">CLMS HRSI User Section website.</a>

Now, we can use the same function to filter or use the GF layer according to different information from QCFLAGS. First, we can separate the GF raster according to sensor type. If Bit 6 is 1, then the pixel is from radar (S1) and if it is 0, then the sensor is optical (S2). We can create rasters and replace filtered values with no data value (255):

In [ ]:
gf_optical = deepcopy(productData['GF'])
gf_radar = deepcopy(productData['GF'])
np.place(gf_radar, ~getBit(productData['QCFLAGS'],6), 255)
np.place(gf_optical, getBit(productData['QCFLAGS'],6), 255)

Draw the separated rasters together using matplotlib:

In [ ]:
plt.subplot(1, 3, 1)
plt.title("GF")
plt.imshow(
    productData['GF'], cmap = ListedColormap(productData['GF_cmp']),interpolation = 'nearest'
);
plt.clim(0, 255)
plt.xticks([]);
plt.yticks([]);

plt.subplot(1, 3, 2)
plt.title("Optical only")
plt.imshow(
    gf_optical, cmap = ListedColormap(productData['GF_cmp']),interpolation = 'nearest'
);
plt.clim(0, 255)
plt.xticks([]);
plt.yticks([]);

plt.subplot(1, 3, 3)
plt.title("Radar only")
plt.imshow(
    gf_radar, cmap = ListedColormap(productData['GF_cmp']),interpolation = 'nearest'
);
plt.clim(0, 255)
plt.xticks([]);
plt.yticks([]);

<h4>One last example</h4>
We can merge information from AT and QCFLAGS layers to filter pixels from GF layer, for example only from optical sensor and not older than 2 days:
<li>"invalid value encountered" warning will appear due to np.nan in age, but result is false for those pixels. Thus, they won't disturb the result.

In [ ]:
gf_day_optical = deepcopy(productData['GF'])
np.place(gf_day_optical, getBit(productData['QCFLAGS'],6), 255)
np.place(gf_day_optical, productData['age'] > 1, 255)

Draw the GF and filtered GF using matplotlib:

In [ ]:
plt.subplot(1, 2, 1)
plt.title("GF")
plt.imshow(
    productData['GF'], cmap = ListedColormap(productData['GF_cmp']), interpolation = 'nearest'
);
plt.clim(0, 255)
plt.xticks([]);
plt.yticks([]);

plt.subplot(1, 2, 2)
plt.title("1 day  Optical Only")
plt.imshow(
    gf_day_optical, cmap = ListedColormap(productData['GF_cmp']), interpolation = 'nearest'
);
plt.clim(0, 255)
plt.xticks([]);
plt.yticks([]);